In [1]:

import torch
from data_provider.data_factory import data_provider
from models import PatchTST
import importlib
importlib.reload(PatchTST)


<module 'models.PatchTST' from 'c:\\Users\\Omen\\PycharmProjects\\pointdata\\PatchTST_supervised\\models\\PatchTST.py'>

In [33]:
class configClass():
    activation='gelu'
    affine=0
    batch_size=128
    c_out=7
    checkpoints='./checkpoints/'
    d_ff=256
    d_layers=1
    d_model=128
    data='custom'
    data_path='weather.csv'
    dec_in=7
    decomposition=0
    des='Exp'
    devices='0,1,2,3'
    distil=True
    do_predict=False
    dropout=0.2
    e_layers=3
    embed='timeF'
    embed_type=0
    enc_in=21
    factor=1
    fc_dropout=0.2
    features='M'
    freq='h'
    gpu=0
    head_dropout=0.0
    individual=0
    is_training=1
    itr=1
    kernel_size=25
    label_len=48
    learning_rate=0.0001
    loss='mse'
    lradj='type3'
    model='PatchTST'
    model_id='model_id'
    moving_avg=25
    n_heads=16
    num_workers=10
    output_attention=False
    padding_patch='end'
    patch_len=16
    patience=20
    pct_start=0.3
    pred_len=96
    profiler_logdir='log/profiler_dir'
    random_seed=2021
    revin=1
    root_path='./dataset/'
    seq_len=336
    stride=8
    subtract_last=0
    target='OT'
    test_flop=False
    train_epochs=100
    use_amp=False
    use_flashattention=False
    use_gpu=True
    use_multi_gpu=False
config = configClass()

In [34]:
# data_set, data_loader = data_provider(config, "train")
# test_batch = next(iter(data_loader))

In [44]:
model = PatchTST.Model(config).cuda()

batch_rand = torch.randn(config.batch_size, config.seq_len, 5).cuda()

In [45]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params


921184

In [46]:
outputs = model(batch_rand)

not using flash attention
not using flash attention
not using flash attention


In [47]:
outputs.shape

torch.Size([128, 96, 5])

In [6]:
config.use_flashattention = True
model = PatchTST.Model(config).cuda()


In [7]:
model

Model(
  (model): PatchTST_backbone(
    (revin_layer): RevIN()
    (padding_patch_layer): ReplicationPad1d((0, 8))
    (backbone): TSTiEncoder(
      (W_P): Linear(in_features=16, out_features=128, bias=True)
      (dropout): Dropout(p=0.2, inplace=False)
      (encoder): TSTEncoder(
        (layers): ModuleList(
          (0-2): 3 x TSTEncoderLayer(
            (self_attn): _MultiheadAttention(
              (W_Q): Linear(in_features=128, out_features=128, bias=True)
              (W_K): Linear(in_features=128, out_features=128, bias=True)
              (W_V): Linear(in_features=128, out_features=128, bias=True)
              (sdp_attn): _ScaledDotProductAttention(
                (attn_dropout): Dropout(p=0.0, inplace=False)
              )
              (to_out): Sequential(
                (0): Linear(in_features=128, out_features=128, bias=True)
                (1): Dropout(p=0.2, inplace=False)
              )
            )
            (dropout_attn): Dropout(p=0.2, inplace=Fals

In [ ]:
outputs = model(batch_rand)

In [ ]:
outputs.shape

In [2]:
import pandas as pd
df_raw = pd.read_csv(r"C:\Users\Omen\PycharmProjects\pointdata\PatchTST_supervised\dataset\weather.csv")


In [16]:
cols = list(df_raw.columns)
cols.remove('OT')
cols.remove('date')
df_raw = df_raw[['date'] + cols + ['OT']]

In [21]:
df_raw.columns

Index(['date', 'p (mbar)', 'T (degC)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)',
       'VPmax (mbar)', 'VPact (mbar)', 'VPdef (mbar)', 'sh (g/kg)',
       'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)', 'max. wv (m/s)',
       'wd (deg)', 'rain (mm)', 'raining (s)', 'SWDR (W/m�)',
       'PAR (�mol/m�/s)', 'max. PAR (�mol/m�/s)', 'Tlog (degC)', 'OT'],
      dtype='object')

In [19]:

cols_data = df_raw.columns[1:]

In [22]:
df_data = df_raw[cols_data]

In [28]:
df_data.values.shape

(52696, 21)

In [29]:
from data_provider.data_loader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_Pred

data_set = Dataset_Custom(
        root_path=config.root_path,
        data_path=config.data_path,
        flag="train",
        size=[config.seq_len, config.label_len, config.pred_len],
        features=config.features,
        target=config.target,
        timeenc=1,
        freq=config.freq
    )

In [30]:
seq_x, seq_y, seq_x_mark, seq_y_mark = data_set[0]

In [31]:
seq_x.shape

(1024, 21)

In [41]:
torch.cuda.empty_cache()